In [ ]:
import os
import re
import random
from datetime import datetime

from pathlib import Path
os.chdir('/content/drive/MyDrive/datadriven/airathon')
DATA_PATH = Path.cwd() / 'data'
RAW = DATA_PATH / 'raw'
PROCESSED = DATA_PATH / 'processed'

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = (16, 8)

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import tensorflow as tf
from tensorflow import keras
from keras import layers

from tqdm import tqdm

In [ ]:
train, test = {}, {}
locations = ['la', 'dl', 'tpe']

for loc in locations:
    train[loc] = pd.read_csv(str(PROCESSED / f'train_{loc}.csv'), parse_dates=['datetime', 'day']).sort_values(by=['datetime'])
    test[loc] = pd.read_csv(str(PROCESSED / f'test_{loc}.csv'), parse_dates=['datetime', 'day']).sort_values(by=['datetime'])
    

data = pd.concat([train['la'], train['dl'], train['tpe']]).dropna()
test_data = pd.concat([test['la'], test['dl'], test['tpe']])

target = np.array(data.pm25).reshape(-1,1)
target_scaler = MinMaxScaler()
target = target_scaler.fit_transform(target)

data.drop(columns=['datetime', 'pm25', 'day'], axis=1, inplace=True)
test_data.drop(columns=['datetime', 'value', 'day'], axis=1, inplace=True)

ohe_features = ['grid_id']
ohe_transformer = OneHotEncoder(handle_unknown='ignore')

num_features = ['Optical_Depth_047_mean',
       'Optical_Depth_047_min', 'Optical_Depth_047_max',
       'Optical_Depth_047_std', 'Optical_Depth_055_mean',
       'Optical_Depth_055_min', 'Optical_Depth_055_max',
       'Optical_Depth_055_std', 'AOD_Uncertainty_mean', 'AOD_Uncertainty_min',
       'AOD_Uncertainty_max', 'AOD_Uncertainty_std', 'Column_WV_mean',
       'Column_WV_min', 'Column_WV_max', 'Column_WV_std', 'cosSZA_mean',
       'cosSZA_min', 'cosSZA_max', 'cosSZA_std', 'cosVZA_mean', 'cosVZA_min',
       'cosVZA_max', 'cosVZA_std', 'RelAZ_mean', 'RelAZ_min', 'RelAZ_max',
       'RelAZ_std', 'Scattering_Angle_mean', 'Scattering_Angle_min',
       'Scattering_Angle_max', 'Scattering_Angle_std', 'Glint_Angle_mean',
       'Glint_Angle_min', 'Glint_Angle_max', 'Glint_Angle_std']

num_transformer = Pipeline(steps=[
                                  ('imputer', SimpleImputer(strategy='mean')), 
                                  ('scaler', MinMaxScaler())
                                  ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', ohe_transformer, ohe_features)])

data = preprocessor.fit_transform(data)
test_data = preprocessor.transform(test_data)

n_features = data.shape[1]




In [ ]:
model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[n_features]),
    layers.Dropout(.3),
    layers.Dense(60, activation=tf.nn.relu),
    layers.Dropout(.3),
    layers.Dense(56, activation=tf.nn.relu),
    layers.Dropout(.3),
    layers.Dense(52, activation=tf.nn.relu),
    layers.Dropout(.3),
    layers.Dense(48, activation=tf.nn.relu),
    layers.Dropout(.3),
    layers.Dense(44, activation=tf.nn.relu),
    layers.Dropout(.3),
    layers.Dense(40, activation=tf.nn.relu),
    layers.Dropout(.3),
    layers.Dense(36, activation=tf.nn.relu),
    layers.Dropout(.3),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dropout(.3),
    layers.Dense(28, activation=tf.nn.relu),
    layers.Dropout(.3),
    layers.Dense(24, activation=tf.nn.relu),
    layers.Dropout(.3),
    layers.Dense(20, activation=tf.nn.relu),
    layers.Dropout(.3),
    layers.Dense(10, activation=tf.nn.relu),
    layers.Dense(1)
    ])

optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 64)                5120      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_30 (Dense)            (None, 60)                3900      
                                                                 
 dropout_1 (Dropout)         (None, 60)                0         
                                                                 
 dense_31 (Dense)            (None, 56)                3416      
                                                                 
 dropout_2 (Dropout)         (None, 56)                0         
                                                                 
 dense_32 (Dense)            (None, 52)               

In [ ]:
model.predict(data[0:10])

array([[0.00040925],
       [0.00066063],
       [0.00021395],
       [0.00038469],
       [0.00023013],
       [0.00031306],
       [0.0013245 ],
       [0.00035073],
       [0.00052658],
       [0.00070483]], dtype=float32)

In [ ]:
EPOCHS = 100
batch_size = 64 

history = model.fit(data, target.reshape(-1,), epochs=EPOCHS, validation_split=0.2, batch_size=batch_size)

Epoch 1/100
196/196 [==============================] - 2s 6ms/step - loss: 0.0068 - mse: 0.0068 - val_loss: 0.0145 - val_mse: 0.0145
Epoch 2/100
196/196 [==============================] - 1s 4ms/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.0119 - val_mse: 0.0119
Epoch 3/100
196/196 [==============================] - 1s 4ms/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.0104 - val_mse: 0.0104
Epoch 4/100
196/196 [==============================] - 1s 4ms/step - loss: 0.0033 - mse: 0.0033 - val_loss: 0.0096 - val_mse: 0.0096
Epoch 5/100
196/196 [==============================] - 1s 4ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.0088 - val_mse: 0.0088
Epoch 6/100
196/196 [==============================] - 1s 4ms/step - loss: 0.0029 - mse: 0.0029 - val_loss: 0.0102 - val_mse: 0.0102
Epoch 7/100
196/196 [==============================] - 1s 4ms/step - loss: 0.0027 - mse: 0.0027 - val_loss: 0.0096 - val_mse: 0.0096
Epoch 8/100
196/196 [==============================] - 1s 4ms/step - 

In [ ]:
pred = model.predict(test_data)
pred = target_scaler.inverse_transform(pred)

test_df = pd.concat([test['la'], test['dl'], test['tpe']])
test_df.value = pred

submission = pd.read_csv(RAW / 'submission_format.csv', parse_dates=['datetime'])

In [ ]:
for i in tqdm(range(len(submission))):
    sample = submission.iloc[i]
    if len(test_df[(test_df.datetime == sample.datetime) & (test_df.grid_id == sample.grid_id)]) != 1:
        print(len(test_df[(test_df.datetime == sample.datetime) & (test_df.grid_id == sample.grid_id)]))
        break 
        
    submission.loc[i, 'value'] = test_df[(test_df.datetime == sample.datetime) & (test_df.grid_id == sample.grid_id)].value.values[0]

100%|██████████| 13504/13504 [00:49<00:00, 274.18it/s]


In [ ]:
submission_format = pd.read_csv(RAW / 'submission_format.csv')
submission_format.value = submission.value
submission_format.to_csv(DATA_PATH / 'submission/feed_foward.csv', index=False)

In [ ]:
WLSV_5011 = pd.read_csv(DATA_PATH / 'submission/WLSV_5011.csv')
feed_forward = pd.read_csv(DATA_PATH / 'submission/feed_forward.csv')
submission_format = pd.read_csv(RAW / 'submission_format.csv')
submission_format.value = (2/3)*(WLSV_5011.value) + (1/3)*(feed_forward.value)
submission_format.to_csv(DATA_PATH / 'submission/WLSV23_feed_fwd13.csv', index=False)